## General Definitions

In [1]:
final case class Fix[F[_]](unFix: F[Fix[F]])
// Fix[F[_]] = F[F[F[...]]]

final case class Fix2[F[_,_],G[_,_]](unFix2: F[Fix2[F,G], Fix2[G,F]])
// Fix2[F[_,_],G[_,_]] = F[
//   F[F[...],G[...]]
//   G[G[...],F[...]]
// ]

type Id = Int

defined class Fix
defined class Fix2
defined type Id

## Non-recursive Example: Schema

In [2]:
sealed trait Ob:
  val id: Id
  var name: String

sealed trait Hom[TSrc,TTgt]:
  val id: Id
  var name: String
  var src: TSrc
  var tgt: TTgt

case class EntityType(
  val id: Id,
  var name: String,
) extends Ob

case class AttributeType(
  val id: Id,
  var name: String,
) extends Ob

case class Relationship[TSrc,TTgt](
  val id: Id,
  var name: String,
  var src: TSrc,
  var tgt: TTgt,
) extends Hom[TSrc,TTgt]

case class Attribute[TSrc,TTgt](
  val id: Id,
  var name: String,
  var src: TSrc,
  var tgt: TTgt,
) extends Hom[TSrc,TTgt]

case class Database[TEntityType, TAttributeType, TRelationship, TAttribute](
  val entityTypes: Seq[TEntityType],
  val attributeTypes: Seq[TAttributeType],
  val relationships: Seq[TRelationship],
  val attributes: Seq[TAttribute],
)

defined trait Ob
defined trait Hom
defined class EntityType
defined class AttributeType
defined class Relationship
defined class Attribute
defined class Database

In [3]:
type DBF = Database[
    EntityType,
    AttributeType,
    Relationship[Id, Id],
    Attribute[Id, Id]
]
type DBN = Database[
    EntityType,
    AttributeType,
    Relationship[EntityType, EntityType],
    Attribute[EntityType, AttributeType]
]
def nest(db: DBF): DBN =
  def nestRelationships(relationships: Seq[Relationship[Id, Id]]): Seq[Relationship[EntityType, EntityType]] = 
    relationships.map(relationship => relationship.copy(
      src=db.entityTypes.find(_.id == relationship.src).get,
      tgt=db.entityTypes.find(_.id == relationship.tgt).get,
    ))
  def nestAttributes(attributes: Seq[Attribute[Id, Id]]): Seq[Attribute[EntityType, AttributeType]] = 
    attributes.map(attribute => attribute.copy(
      src=db.entityTypes.find(_.id == attribute.src).get,
      tgt=db.attributeTypes.find(_.id == attribute.tgt).get,
    ))
  db.copy(
    relationships=nestRelationships(db.relationships),
    attributes=nestAttributes(db.attributes),
  )
def unnest(db: DBN): DBF =
  def unnestRelationships(relationships: Seq[Relationship[EntityType, EntityType]]): Seq[Relationship[Id, Id]] = 
    relationships.map(relationship => relationship.copy(
      src=relationship.src.id,
      tgt=relationship.tgt.id,
    ))
  def unnestAttributes(attributes: Seq[Attribute[EntityType, AttributeType]]): Seq[Attribute[Id, Id]] = 
    attributes.map(attribute => attribute.copy(
      src=attribute.src.id,
      tgt=attribute.tgt.id,
    ))
  db.copy(
    relationships=unnestRelationships(db.relationships),
    attributes=unnestAttributes(db.attributes),
  )

defined type DBF
defined type DBN
defined function nest
defined function unnest

In [4]:
val dbf = Database(
  Seq(EntityType(0, "E"), EntityType(1, "V")),
  Seq(AttributeType(0, "Number")),
  Seq(Relationship(0, "src", 0, 1), Relationship(0, "tgt", 0, 1)),
  Seq(Attribute(0, "weight", 0, 0)),
)

dbf: Database[EntityType, AttributeType, Relationship[Int, Int], Attribute[Int, Int]] = Database(
  entityTypes = List(
    EntityType(id = 0, name = "E"),
    EntityType(id = 1, name = "V")
  ),
  attributeTypes = List(AttributeType(id = 0, name = "Number")),
  relationships = List(
    Relationship(id = 0, name = "src", src = 0, tgt = 1),
    Relationship(id = 0, name = "tgt", src = 0, tgt = 1)
  ),
  attributes = List(Attribute(id = 0, name = "weight", src = 0, tgt = 0))
)

In [5]:
val dbn = nest(dbf)

dbn: Database[EntityType, AttributeType, Relationship[EntityType, EntityType], Attribute[EntityType, AttributeType]] = Database(
  entityTypes = List(
    EntityType(id = 0, name = "E"),
    EntityType(id = 1, name = "V")
  ),
  attributeTypes = List(AttributeType(id = 0, name = "Number")),
  relationships = List(
    Relationship(
      id = 0,
      name = "src",
      src = EntityType(id = 0, name = "E"),
      tgt = EntityType(id = 1, name = "V")
    ),
    Relationship(
      id = 0,
      name = "tgt",
      src = EntityType(id = 0, name = "E"),
      tgt = EntityType(id = 1, name = "V")
    )
  ),
  attributes = List(
    Attribute(
      id = 0,
      name = "weight",
      src = EntityType(id = 0, name = "E"),
      tgt = AttributeType(id = 0, name = "Number")
    )
  )
)

In [6]:
val a = dbn.attributes(0)
s"${a.name} :: ${a.src.name} -> ${a.tgt.name}"

a: Attribute[EntityType, AttributeType] = Attribute(
  id = 0,
  name = "weight",
  src = EntityType(id = 0, name = "E"),
  tgt = AttributeType(id = 0, name = "Number")
)
res5_1: String = "weight :: E -> Number"

In [7]:
a.tgt.name = "Float32"

In [8]:
unnest(dbn)

res7: Database[EntityType, AttributeType, Relationship[Id, Id], Attribute[Id, Id]] = Database(
  entityTypes = List(
    EntityType(id = 0, name = "E"),
    EntityType(id = 1, name = "V")
  ),
  attributeTypes = List(AttributeType(id = 0, name = "Float32")),
  relationships = List(
    Relationship(id = 0, name = "src", src = 0, tgt = 1),
    Relationship(id = 0, name = "tgt", src = 0, tgt = 1)
  ),
  attributes = List(Attribute(id = 0, name = "weight", src = 0, tgt = 0))
)

## Recursive Example: Nodes with parent foreign key/reference.

Types and recursion scheme/fixpoint boilerplate.

In [9]:
case class NodeF[TValue, TParent](
  val id: Id,
  var parent: TParent,
  var value: TValue,
)

type Node[TValue] = Fix[[TParent] =>> NodeF[TValue, TParent]]

object Node:
  def apply[TValue](id: Id, parent: Node[TValue], value: TValue): Node[TValue] =
    Fix(NodeF(id, parent, value))

case class Database[TNode](
  val nodes: Seq[TNode],
)

defined class NodeF
defined type Node
defined object Node
defined class Database

Convert between unnested and nested representation:

In [10]:
type DBF = [TValue] =>> Database[
  NodeF[TValue, Id]
]
type DBN = [TValue] =>> Database[
  Node[TValue]
]

def nest[TValue](db: DBF[TValue]): DBN[TValue] =
  def nestParent[TValue](nodes: Seq[NodeF[TValue, Id]]): Seq[Node[TValue]] = 
    val cache = Map.from(nodes.map(node => 
      (node.id, Fix(node.copy(parent=null)))
    ))
    nodes.map(node => {
      val Fix(nodeF) = cache(node.id)
      nodeF.parent = cache(node.parent)
      Fix(nodeF)
    })
  db.copy(nodes=nestParent(db.nodes))

def unnest[TValue](db: DBN[TValue]): DBF[TValue] =
  def unnestParent[TValue](nodes: Seq[Node[TValue]]): Seq[NodeF[TValue, Id]] =
    nodes.map(node => 
      val Fix(nodeF) = node
      val Fix(parentF) = nodeF.parent
      nodeF.copy(parent=parentF.id)
    )
  db.copy(nodes=unnestParent(db.nodes))

defined type DBF
defined type DBN
defined function nest
defined function unnest

Flat representation:

In [11]:
val dbf = Database(Seq(
  NodeF(0, 0, "Root"),
  NodeF(1, 0, "A"),
  NodeF(2, 1, "A-1"),
  NodeF(3, 2, "A-2"),
  NodeF(4, 0, "B")
))

dbf: Database[NodeF[String, Int]] = Database(
  nodes = List(
    NodeF(id = 0, parent = 0, value = "Root"),
    NodeF(id = 1, parent = 0, value = "A"),
    NodeF(id = 2, parent = 1, value = "A-1"),
    NodeF(id = 3, parent = 2, value = "A-2"),
    NodeF(id = 4, parent = 0, value = "B")
  )
)

Nested representation:

In [12]:
val dbn = nest(dbf)

dbn: Database[Node[String]] = Database(
  nodes = List(
    Fix(
      unFix = NodeF(
        id = 0,
        parent = Fix(
          unFix = NodeF(
            id = 0,
            parent = Fix(
              unFix = NodeF(
                id = 0,
                parent = Fix(
                  unFix = NodeF(
                    id = 0,
                    parent = Fix(
                      unFix = NodeF(
                        id = 0,
                        parent = Fix(
                          unFix = NodeF(
                            id = 0,
                            parent = Fix(
                              unFix = NodeF(
                                id = 0,
                                parent = Fix(
                                  unFix = NodeF(
                                    id = 0,
                                    parent = Fix(
                                      unFix = NodeF(
                                        id = 0,
                          

Work with nested representation:

In [13]:
def path[TValue](node: Node[TValue]): Seq[TValue] = 
  val Fix(nodeF) = node
  val Fix(parentF) = nodeF.parent
  val parentPath = if nodeF.id == parentF.id then Nil else path(nodeF.parent)
  parentPath :+ nodeF.value

defined function path

In [14]:
path(dbn.nodes(2))

res13: Seq[String] = List("Root", "A", "A-1")

Updated nested representation:

In [15]:
val dbn2 = dbn.copy(nodes=dbn.nodes :+ Node(5, dbn.nodes(0), "C"))

dbn2: Database[Fix[ammonite.$sess.cmd9.wrapper.cmd8.Database[ammonite.$sess.cmd8.wrapper.cmd0.Fix[[TParent >: scala.Nothing <: scala.Any] => ammonite.$sess.cmd9.wrapper.cmd8.NodeF[java.lang.String, TParent]]]]] = Database(
  nodes = List(
    Fix(
      unFix = NodeF(
        id = 0,
        parent = Fix(
          unFix = NodeF(
            id = 0,
            parent = Fix(
              unFix = NodeF(
                id = 0,
                parent = Fix(
                  unFix = NodeF(
                    id = 0,
                    parent = Fix(
                      unFix = NodeF(
                        id = 0,
                        parent = Fix(
                          unFix = NodeF(
                            id = 0,
                            parent = Fix(
                              unFix = NodeF(
                                id = 0,
                                parent = Fix(
                                  unFix = NodeF(
                                    id

Updated flat representation:

In [16]:
val dbf2 = unnest(dbn2)

dbf2: Database[NodeF[String, Id]] = Database(
  nodes = List(
    NodeF(id = 0, parent = 0, value = "Root"),
    NodeF(id = 1, parent = 0, value = "A"),
    NodeF(id = 2, parent = 1, value = "A-1"),
    NodeF(id = 3, parent = 2, value = "A-2"),
    NodeF(id = 4, parent = 0, value = "B"),
    NodeF(id = 5, parent = 0, value = "C")
  )
)

## Mutually Recursive Example: Employee/Department

In [17]:
case class Person[TDepartment](
  val id: Id,
  var name: String,
  var worksIn: TDepartment,
)

case class Department[TPerson](
  val id: Id,
  var name: String,
  var manager: TPerson,
)

case class Database[TPerson, TDepartment](
  val persons: Seq[TPerson],
  val departments: Seq[TDepartment],
)

defined class Person
defined class Department
defined class Database

In [18]:
type PersonF = Person[Id]
type DepartmentF = Department[Id]

type PersonN = Fix2[
  [TPerson, TDepartment] =>> Person[TDepartment],
  [TPerson, TDepartment] =>> Department[TDepartment]
]
type DepartmentN = Fix2[
  [TDepartment, TPerson] =>> Department[TPerson],
  [TDepartment, TPerson] =>> Person[TPerson]
]

object PersonN:
  def apply(id: Id, name: String, worksIn: DepartmentN): PersonN =
    Fix2(Person(id, name, worksIn))

object DepartmentN:
  def apply(id: Id, name: String, manager: PersonN): DepartmentN =
    Fix2(Department(id, name, manager))

type DBF = Database[PersonF, DepartmentF]
type DBN = Database[PersonN, DepartmentN]

defined type PersonF
defined type DepartmentF
defined type PersonN
defined type DepartmentN
defined object PersonN
defined object DepartmentN
defined type DBF
defined type DBN

In [19]:
def nest(db: DBF): DBN =
  val cachePersons: Map[Id, PersonN] = Map.from(db.persons.map(person => 
    (person.id, Fix2(person.copy(worksIn=null)))
  ))
  val cacheDepartments: Map[Id, DepartmentN] = Map.from(db.departments.map(department => 
    (department.id, Fix2(department.copy(manager=null)))
  ))
  def nestPersons(persons: Seq[PersonF]): Seq[PersonN] = 
    persons.map(person => {
      val Fix2(personF) = cachePersons(person.id)
      personF.worksIn = cacheDepartments(person.worksIn)
      Fix2(personF)
    })
  def nestDepartments(departments: Seq[DepartmentF]): Seq[DepartmentN] = 
    departments.map(department => {
      val Fix2(departmentF) = cacheDepartments(department.id)
      departmentF.manager = cachePersons(department.manager)
      Fix2(departmentF)
    })
  db.copy(
    persons=nestPersons(db.persons),
    departments=nestDepartments(db.departments),
  )

def unnest(db: DBN): DBF =
  def unnestPersons(persons: Seq[PersonN]): Seq[PersonF] =
    persons.map(person => 
      val Fix2(personF) = person
      val Fix2(worksInF) = personF.worksIn
      personF.copy(worksIn=worksInF.id)
    )
  def unnestDepartments(departments: Seq[DepartmentN]): Seq[DepartmentF] =
    departments.map(department => 
      val Fix2(departmentF) = department
      val Fix2(managerF) = departmentF.manager
      departmentF.copy(manager=managerF.id)
    )
  db.copy(
    persons=unnestPersons(db.persons),
    departments=unnestDepartments(db.departments),
  )

defined function nest
defined function unnest

In [20]:
val dbf: DBF = Database(
  Seq(
    Person(0, "Alice", 0),
    Person(1, "Bob", 1),
    Person(2, "Carol", 1),
  ),
  Seq(
    Department(0, "A", 0),
    Department(1, "B", 1),
  )
)

val dbn: DBN = nest(dbf)

dbf: Database[PersonF, DepartmentF] = Database(
  persons = List(
    Person(id = 0, name = "Alice", worksIn = 0),
    Person(id = 1, name = "Bob", worksIn = 1),
    Person(id = 2, name = "Carol", worksIn = 1)
  ),
  departments = List(
    Department(id = 0, name = "A", manager = 0),
    Department(id = 1, name = "B", manager = 1)
  )
)
dbn: Database[PersonN, DepartmentN] = Database(
  persons = List(
    Fix2(
      unFix2 = Person(
        id = 0,
        name = "Alice",
        worksIn = Fix2(
          unFix2 = Department(
            id = 0,
            name = "A",
            manager = Fix2(
              unFix2 = Person(
                id = 0,
                name = "Alice",
                worksIn = Fix2(
                  unFix2 = Department(
                    id = 0,
                    name = "A",
                    manager = Fix2(
                      unFix2 = Person(
                        id = 0,
                        name = "Alice",
                        works

In [21]:
for (person <- dbn.persons)
  val Fix2(personF) = person
  val Fix2(departmentF) = personF.worksIn
  val Fix2(managerF) = departmentF.manager
  println(s"${personF.name} (${departmentF.name}, ${managerF.name})")

Alice (A, Alice)
Bob (B, Bob)
Carol (B, Bob)


**Idea: Implement a generic way to do this for any/all homs/foreign keys?**